This notebook constructs a more complex model of evolving state number densities for a multiple mass bins.  The method is tested by simulating some data and fitting it, recovering the correct parameters.  The fit may be performed using direct minimization, or MCMC in order to explore the parameter space.

In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
from scipy.integrate import odeint
from astropy import cosmology
from IPython.display import Image
import pickle

In [2]:
# Steven's emcee stuff

from convenience import run_emcee
from pdfs import norm_logpdf, beta_logpdf
import plots

In [3]:
%matplotlib inline
# better-looking plots
plt.rcParams['font.family'] = 'serif'
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['font.size'] = 12

In [4]:
C = cosmology.FlatLambdaCDM(70.0, 0.3)

In [5]:
basic_states = ['BD', 'RD', 'RE']
states = []
for i in range(6):
    for s in basic_states:
        states.append('{}{}'.format(s, i))
states.append('NE')
states = pd.Index(states)
nStates = len(states)

In [6]:
states

Index(['BD0', 'RD0', 'RE0', 'BD1', 'RD1', 'RE1', 'BD2', 'RD2', 'RE2', 'BD3',
       'RD3', 'RE3', 'BD4', 'RD4', 'RE4', 'BD5', 'RD5', 'RE5', 'NE'],
      dtype='object')

In [7]:
rate_labels = [['$r_{' + '{} \\rightarrow {}'.format(s2, s1) + '}$' for s2 in states] for s1 in states]
rate_labels = pd.DataFrame(index='to ' + states, columns='from ' + states, data=rate_labels)
rate_evol_labels = rate_labels.replace('\$r', '$\dot{r}', regex=True)
N_labels = '$N_{' + states + '}$'

In [8]:
N_labels

Index(['$N_{BD0}$', '$N_{RD0}$', '$N_{RE0}$', '$N_{BD1}$', '$N_{RD1}$',
       '$N_{RE1}$', '$N_{BD2}$', '$N_{RD2}$', '$N_{RE2}$', '$N_{BD3}$',
       '$N_{RD3}$', '$N_{RE3}$', '$N_{BD4}$', '$N_{RD4}$', '$N_{RE4}$',
       '$N_{BD5}$', '$N_{RD5}$', '$N_{RE5}$', '$N_{NE}$'],
      dtype='object')

In [9]:
rate_labels

,from BD0,from RD0,from RE0,from BD1,from RD1,from RE1,from BD2,from RD2,from RE2,from BD3,from RD3,from RE3,from BD4,from RD4,from RE4,from BD5,from RD5,from RE5,from NE
to BD0,$r_{BD0 \rightarrow BD0}$,$r_{RD0 \rightarrow BD0}$,$r_{RE0 \rightarrow BD0}$,$r_{BD1 \rightarrow BD0}$,$r_{RD1 \rightarrow BD0}$,$r_{RE1 \rightarrow BD0}$,$r_{BD2 \rightarrow BD0}$,$r_{RD2 \rightarrow BD0}$,$r_{RE2 \rightarrow BD0}$,$r_{BD3 \rightarrow BD0}$,$r_{RD3 \rightarrow BD0}$,$r_{RE3 \rightarrow BD0}$,$r_{BD4 \rightarrow BD0}$,$r_{RD4 \rightarrow BD0}$,$r_{RE4 \rightarrow BD0}$,$r_{BD5 \rightarrow BD0}$,$r_{RD5 \rightarrow BD0}$,$r_{RE5 \rightarrow BD0}$,$r_{NE \rightarrow BD0}$
to RD0,$r_{BD0 \rightarrow RD0}$,$r_{RD0 \rightarrow RD0}$,$r_{RE0 \rightarrow RD0}$,$r_{BD1 \rightarrow RD0}$,$r_{RD1 \rightarrow RD0}$,$r_{RE1 \rightarrow RD0}$,$r_{BD2 \rightarrow RD0}$,$r_{RD2 \rightarrow RD0}$,$r_{RE2 \rightarrow RD0}$,$r_{BD3 \rightarrow RD0}$,$r_{RD3 \rightarrow RD0}$,$r_{RE3 \rightarrow RD0}$,$r_{BD4 \rightarrow RD0}$,$r_{RD4 \rightarrow RD0}$,$r_{RE4 \rightarrow RD0}$,$r_{BD5 \rightarrow RD0}$,$r_{RD5 \rightarrow RD0}$,$r_{RE5 \rightarrow RD0}$,$r_{NE \rightarrow RD0}$
to RE0,$r_{BD0 \rightarrow RE0}$,$r_{RD0 \rightarrow RE0}$,$r_{RE0 \rightarrow RE0}$,$r_{BD1 \rightarrow RE0}$,$r_{RD1 \rightarrow RE0}$,$r_{RE1 \rightarrow RE0}$,$r_{BD2 \rightarrow RE0}$,$r_{RD2 \rightarrow RE0}$,$r_{RE2 \rightarrow RE0}$,$r_{BD3 \rightarrow RE0}$,$r_{RD3 \rightarrow RE0}$,$r_{RE3 \rightarrow RE0}$,$r_{BD4 \rightarrow RE0}$,$r_{RD4 \rightarrow RE0}$,$r_{RE4 \rightarrow RE0}$,$r_{BD5 \rightarrow RE0}$,$r_{RD5 \rightarrow RE0}$,$r_{RE5 \rightarrow RE0}$,$r_{NE \rightarrow RE0}$
to BD1,$r_{BD0 \rightarrow BD1}$,$r_{RD0 \rightarrow BD1}$,$r_{RE0 \rightarrow BD1}$,$r_{BD1 \rightarrow BD1}$,$r_{RD1 \rightarrow BD1}$,$r_{RE1 \rightarrow BD1}$,$r_{BD2 \rightarrow BD1}$,$r_{RD2 \rightarrow BD1}$,$r_{RE2 \rightarrow BD1}$,$r_{BD3 \rightarrow BD1}$,$r_{RD3 \rightarrow BD1}$,$r_{RE3 \rightarrow BD1}$,$r_{BD4 \rightarrow BD1}$,$r_{RD4 \rightarrow BD1}$,$r_{RE4 \rightarrow BD1}$,$r_{BD5 \rightarrow BD1}$,$r_{RD5 \rightarrow BD1}$,$r_{RE5 \rightarrow BD1}$,$r_{NE \rightarrow BD1}$
to RD1,$r_{BD0 \rightarrow RD1}$,$r_{RD0 \rightarrow RD1}$,$r_{RE0 \rightarrow RD1}$,$r_{BD1 \rightarrow RD1}$,$r_{RD1 \rightarrow RD1}$,$r_{RE1 \rightarrow RD1}$,$r_{BD2 \rightarrow RD1}$,$r_{RD2 \rightarrow RD1}$,$r_{RE2 \rightarrow RD1}$,$r_{BD3 \rightarrow RD1}$,$r_{RD3 \rightarrow RD1}$,$r_{RE3 \rightarrow RD1}$,$r_{BD4 \rightarrow RD1}$,$r_{RD4 \rightarrow RD1}$,$r_{RE4 \rightarrow RD1}$,$r_{BD5 \rightarrow RD1}$,$r_{RD5 \rightarrow RD1}$,$r_{RE5 \rightarrow RD1}$,$r_{NE \rightarrow RD1}$
to RE1,$r_{BD0 \rightarrow RE1}$,$r_{RD0 \rightarrow RE1}$,$r_{RE0 \rightarrow RE1}$,$r_{BD1 \rightarrow RE1}$,$r_{RD1 \rightarrow RE1}$,$r_{RE1 \rightarrow RE1}$,$r_{BD2 \rightarrow RE1}$,$r_{RD2 \rightarrow RE1}$,$r_{RE2 \rightarrow RE1}$,$r_{BD3 \rightarrow RE1}$,$r_{RD3 \rightarrow RE1}$,$r_{RE3 \rightarrow RE1}$,$r_{BD4 \rightarrow RE1}$,$r_{RD4 \rightarrow RE1}$,$r_{RE4 \rightarrow RE1}$,$r_{BD5 \rightarrow RE1}$,$r_{RD5 \rightarrow RE1}$,$r_{RE5 \rightarrow RE1}$,$r_{NE \rightarrow RE1}$
to BD2,$r_{BD0 \rightarrow BD2}$,$r_{RD0 \rightarrow BD2}$,$r_{RE0 \rightarrow BD2}$,$r_{BD1 \rightarrow BD2}$,$r_{RD1 \rightarrow BD2}$,$r_{RE1 \rightarrow BD2}$,$r_{BD2 \rightarrow BD2}$,$r_{RD2 \rightarrow BD2}$,$r_{RE2 \rightarrow BD2}$,$r_{BD3 \rightarrow BD2}$,$r_{RD3 \rightarrow BD2}$,$r_{RE3 \rightarrow BD2}$,$r_{BD4 \rightarrow BD2}$,$r_{RD4 \rightarrow BD2}$,$r_{RE4 \rightarrow BD2}$,$r_{BD5 \rightarrow BD2}$,$r_{RD5 \rightarrow BD2}$,$r_{RE5 \rightarrow BD2}$,$r_{NE \rightarrow BD2}$
to RD2,$r_{BD0 \rightarrow RD2}$,$r_{RD0 \rightarrow RD2}$,$r_{RE0 \rightarrow RD2}$,$r_{BD1 \rightarrow RD2}$,$r_{RD1 \rightarrow RD2}$,$r_{RE1 \rightarrow RD2}$,$r_{BD2 \rightarrow RD2}$,$r_{RD2 \rightarrow RD2}$,$r_{RE2 \rightarrow RD2}$,$r_{BD3 \rightarrow RD2}$,$r_{RD3 \rightarrow RD2}$,$r_{RE3 \rightarrow RD2}$,$r_{BD4 \r

In [10]:
# Set initial number densities
N0 = pd.Series(index=N_labels, data=np.zeros(nStates))

# Set rates per galaxy in Gyr^{-1} at fiducial redshift
rates = pd.DataFrame(index='to ' + states, columns='from ' + states,
                     data=np.zeros((nStates, nStates)))

# Modelling rates as linear function of cosmic time
#(for simplicity, should probably choose someting more appropriate)
rates_evol = pd.DataFrame(index='to ' + states, columns='from ' + states,
                     data=np.zeros((nStates, nStates)))

# Select which parameters to fit.
fit_N0 = pd.Series(index=N_labels, data=np.zeros(nStates, np.bool))
fit_rates = pd.DataFrame(index='to ' + states, columns='from ' + states,
                         data=np.zeros((nStates, nStates), np.bool))
#fit_rates_evol = pd.DataFrame(index='to ' + states, columns='from ' + states,
#                         data=np.zeros((nStates, nStates), np.bool))
fit_rates_evol = None

Basic differential equations:

Blue Disks in stellar mass bin $i \in \{1,2,3,4\}$ may:
* spontaneously form in lowest mass bin, at a total rate $f_{BD_1}$
* transition from the previous mass bin (due to SF and minor mergers), at an individual rate $r_{BD_{i-1} \rightarrow BD_{i}}$
* transition to Red Discs in same mass bin, at an individual rate $r_{BD_{i} \rightarrow RD_{i}}$
* transition to Red Ellipticals in same mass bin, at an individual rate $r_{BD_{i} \rightarrow RE_{i}}$
* transition to the next mass bin (due to SF or minor mergers), at an individual rate $r_{BD_{i} \rightarrow BD_{i+1}}$

For convenience we can define $N_{BD_0} = 1$, such that $r_{BD_0 \rightarrow BD_1} = f_{BD_1}$, the spontaneous formation rate.

$\frac{dN_{BD_{i}}}{dt} = + N_{BD_{i-1}} r_{BD_{i-1} \rightarrow BD_{i}} - N_{BD_{i}} \, (r_{BD_{i} \rightarrow RD_{i}} + r_{BD_{i} \rightarrow RE_{i}} + r_{BD_{i} \rightarrow BD_{i+1}})$

Red Disks in stellar mass bin $i \in \{1,2,3,4\}$ may:
* transition from Blue Discs in same mass bin, at an individual rate $r_{BD_{i} \rightarrow RD_{i}}$
* transition to Red Ellipticals in same mass bin, at an individual rate $r_{D_{i} \rightarrow RE_{i}}$

$\frac{dN_{RD_{i}}}{dt} = - N_{RD_{i}} \, r_{RD_{i} \rightarrow RE_{i}} + N_{BD_{i}} \, r_{BD_{i} \rightarrow RD_{i}}$

Red Ellipticals in stellar mass bin $i \in \{1,2,3,4\}$ may:
* transition from the previous mass bin (due to equal mergers), at an individual rate $r_{RE_{i-1} \rightarrow RE_{i}}$
* transition from Blue Discs in same mass bin, at an individual rate $r_{BD_{i} \rightarrow RE_{i}}$
* transition from Red Discs in same mass bin, at an individual rate $r_{RD_{i} \rightarrow RE_{i}}$
* transition to the next mass bin (due to mergers), at an individual rate $r_{RE_{i} \rightarrow RE_{i+1}}$
* transition to None Existent (due to mergers), at an individual rate $r_{RE_{i} \rightarrow NE}$

Note that $N_{RE_0} = 0$ always.


$\frac{dN_{RE_{i}}}{dt} = + N_{RD_{i}} \, r_{RD_{i} \rightarrow RE_{i}} + N_{BD_{i}} \, r_{BD_{i} \rightarrow RE_{i}} + N_{RE_{i-1}} \, r_{RE_{i-1} \rightarrow RE_{i}} - N_{RE_{i}} ( r_{RE_{i} \rightarrow RE_{i+1}} + r_{RE_{i} \rightarrow NE})$ 

The None Existent state and all mass bins beyond the maximum are equivalent.  They do not contribute to the normalisation of the number densities and do need to be tracked.

Must be careful about the number density normalisation – for this to make sense it needs to be done across _all_ states, not just within each mass bin.

In [11]:
fit_N0['$N_{BD0}$'] = False
fit_N0['$N_{RD0}$'] = False
fit_N0['$N_{RE0}$'] = False
fit_N0['$N_{BD1}$'] = True
fit_N0['$N_{RD1}$'] = True
fit_N0['$N_{RE1}$'] = True
fit_N0['$N_{BD2}$'] = True
fit_N0['$N_{RD2}$'] = True
fit_N0['$N_{RE2}$'] = True
fit_N0['$N_{BD3}$'] = True
fit_N0['$N_{RD3}$'] = True
fit_N0['$N_{RE3}$'] = True
fit_N0['$N_{BD4}$'] = True
fit_N0['$N_{RD4}$'] = True
fit_N0['$N_{RE4}$'] = True
fit_N0['$N_{BD5}$'] = False
fit_N0['$N_{RD5}$'] = False
fit_N0['$N_{RE5}$'] = False
fit_N0['$N_{NE}$']  = False

N0['$N_{BD0}$'] = 0
N0['$N_{RD0}$'] = 0
N0['$N_{RE0}$'] = 0
N0['$N_{BD1}$'] = 0.7
N0['$N_{RD1}$'] = 0.1
N0['$N_{RE1}$'] = 0.2
N0['$N_{BD2}$'] = 0.6
N0['$N_{RD2}$'] = 0.1
N0['$N_{RE2}$'] = 0.3
N0['$N_{BD3}$'] = 0.5
N0['$N_{RD3}$'] = 0.2
N0['$N_{RE3}$'] = 0.3
N0['$N_{BD4}$'] = 0.4
N0['$N_{RD4}$'] = 0.2
N0['$N_{RE4}$'] = 0.4
N0['$N_{BD5}$'] = 0
N0['$N_{RD5}$'] = 0
N0['$N_{RE5}$'] = 0
N0['$N_{NE}$']  = 0

N0 /= N0.values.sum()
N0['$N_{BD0}$'] = 1
N0

$N_{BD0}$    1.000
$N_{RD0}$    0.000
$N_{RE0}$    0.000
$N_{BD1}$    0.175
$N_{RD1}$    0.025
$N_{RE1}$    0.050
$N_{BD2}$    0.150
$N_{RD2}$    0.025
$N_{RE2}$    0.075
$N_{BD3}$    0.125
$N_{RD3}$    0.050
$N_{RE3}$    0.075
$N_{BD4}$    0.100
$N_{RD4}$    0.050
$N_{RE4}$    0.100
$N_{BD5}$    0.000
$N_{RD5}$    0.000
$N_{RE5}$    0.000
$N_{NE}$     0.000
dtype: float64

In [12]:
for i in range(5):
    fit_rates['from BD{}'.format(i)]['to BD{}'.format(i+1)] = True

for i in range(1, 5):
    fit_rates['from BD{}'.format(i)]['to RD{}'.format(i)] = True
    fit_rates['from BD{}'.format(i)]['to RE{}'.format(i)] = True
    fit_rates['from RD{}'.format(i)]['to RE{}'.format(i)] = True
    fit_rates['from RE{}'.format(i)]['to NE'] = True

for i in range(1, 6):
    fit_rates['from RE{}'.format(i)]['to RE{}'.format(i+1)] = True

In [ ]:
fit_rates

,from BD0,from RD0,from RE0,from BD1,from RD1,from RE1,from BD2,from RD2,from RE2,from BD3,from RD3,from RE3,from BD4,from RD4,from RE4,from BD5,from RD5,from RE5,from NE
to BD0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
to RD0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
to RE0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
to BD1,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
to RD1,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
to RE1,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
to BD2,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
to RD2,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
to RE2,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False
to BD3,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
def rates_function(rates, rates_evol, t, t0):
    # this used to take pandas DataFrames, but now requires numpy arrays for efficiency
    # shortcut if no evolution
    if rates_evol is None:
        return rates
    rates = rates + rates_evol * (t - t0)
    # * For a closed model the columns of the rates matrix must sum to zero
    #   so there is no creation/destruction of galaxies, this is enforced here.
    # * Creation/destruction of galaxies basically means they move in to or
    #   out of the sample given the selection criteria.
    # * Another possibility is merging, where one galaxies is 'destroyed' in
    #   each merger.
    # * If one wishes to create/destroy galaxies then that could perhaps
    #   be achieved by allowing rates to not sum to zero in each column.
    #   However, the total numbers of galaxies moving between states are
    #   proportional to the number currently in the initial state, which
    #   probably doesn't make much sense for creation.
    #   Alternatively, one could add an additional state(s) representing the pool(s)
    #   from which galaxies are created/destroyed (but this would be evolved itself).
    #   Or one could add additional terms in the differential equation to
    #   describe the creation/destruction of galaxies.
    # * Another issue is multiple selection criteria, e.g. mass bins.
    #   These are essentially different states, and we can model transitions
    #   between them in the same manner: in a small time a galaxy in a given state
    #   will have a particular (potentially time dependent) probability of moving
    #   from one mass bin to the next
    #   (proportional to [SFR + merger accretion rate] / mass_bin_width).
    # * In principle, the rates related to the destruction of galaxies in particular
    #   states due to merging and the mass growth of the merger remnant are related.
    #   In practise this would be pretty complex to manage unless we make some
    #   strict assumptions.  For example, we could assume that only galaxies in the
    #   same state merge (hence ~doubling their mass) and thus require the
    #   destruction rate to be proportional to the mass bin transition rate
    #   (where the factor depends on the width of the mass bin).
    #   If we get too complicated, then it would make more sense to simulate a large
    #   population of individual galaxies rather than consider a small set of states.
    di = np.diag_indices(len(rates))
    rates[di] = 0
    from_sum = rates.sum(0)
    rates[di] = -from_sum
    return rates

In [ ]:
def model(N, t, ratefn, rates, rates_evol, t0):
    rates = ratefn(rates, rates_evol, t, t0)
    return np.dot(rates, N)

In [ ]:
with open('results.pickle', 'rb') as handle:
    results = pickle.load(handle, encoding='latin1')

In [ ]:
mdata = results['m_centers'][1:]
zdata = results['z_centers']
tdata = C.age(zdata)
z0 = 1.0
t0 = C.age(z0).value

In [ ]:
Ndata = pd.DataFrame(index=zdata, columns=N_labels, data=np.zeros((len(zdata), nStates)))
for i in range(1, 5):
    mresults = results['M={:.2f}'.format(mdata[i-1])]
    for key, value in mresults.items():
        if key.startswith('N'):
            state = '{}{}'.format(key[2:], i)
            Ndata['$N_{' + state + '}$'] = value
Ndata = Ndata[::-1]
zdata = zdata[::-1]
tdata = tdata[::-1]
nGal = Ndata.sum(1)

In [ ]:
Ndata

In [ ]:
# strip off unmeasured quantitites
Ndata = Ndata.filter(regex='.*[1234].*')

In [ ]:
Nerr = np.sqrt(Ndata)
Ndata = (Ndata.T / nGal).T
Nerr = (Nerr.T / nGal).T

In [ ]:
Ndata

In [ ]:
Nerr

In [ ]:
def calc_F(N, Nerr=None):
    F = pd.DataFrame(index=N.index)
    if Nerr is not None:
        Ferr = pd.DataFrame(index=N.index)
    for i in range(1, 5):
        m = str(i)
        F['$f_{R|D,' + m + '}$'] = N['$N_{RD' + m + '}$'] / (N['$N_{RD' + m + '}$'] + N['$N_{BD' + m + '}$'])
        F['$f_{D|R,' + m + '}$'] = N['$N_{RD' + m + '}$'] / (N['$N_{RD' + m + '}$'] + N['$N_{RE' + m + '}$'])
        if Nerr is not None:
            NDsum = N['$N_{RD' + m + '}$'] + N['$N_{BD' + m + '}$']
            dfRDdNRD = 1 / NDsum - N['$N_{RD' + m + '}$'] / NDsum**2
            dfRDdNBD = - N['$N_{RD' + m + '}$'] / NDsum**2
            Ferr['$f_{R|D,' + m + '}$'] = np.sqrt(dfRDdNRD**2 * Nerr['$N_{RD' + m + '}$']**2 + 
                                                  dfRDdNBD**2 * Nerr['$N_{BD' + m + '}$']**2)
            NRsum = N['$N_{RD' + m + '}$'] + N['$N_{RE' + m + '}$']
            dfRDdNRD = 1 / NRsum - N['$N_{RD' + m + '}$'] / NRsum**2
            dfRDdNRE = - N['$N_{RD' + m + '}$'] / NRsum**2
            Ferr['$f_{D|R,' + m + '}$'] = np.sqrt(dfRDdNRD**2 * Nerr['$N_{RD' + m + '}$']**2 + 
                                                  dfRDdNRE**2 * Nerr['$N_{RE' + m + '}$']**2)
    if Nerr is not None:
        return F, Ferr
    else:
        return F

In [ ]:
Fdata, Ferr = calc_F(Ndata, Nerr)

In [ ]:
Fdata

In [ ]:
Ferr

In [ ]:
def plot_N(N, Nerr, Nfit=None):
    fig, axarr = plt.subplots(2, 2, figsize=(12, 10))
    axarr = axarr.flat
    for i in range(4):
        Nm = N.filter(regex=('.*{}.*'.format(i+1)))
        Nerrm = Nerr.filter(regex=('.*{}.*'.format(i+1)))
        if Nfit is not None:
            Nfitm = Nfit.filter(regex=('.*{}.*'.format(i+1)))
        for j, col in enumerate(Nm):
            axarr[i].errorbar(Nm.index, Nm[col], Nerrm[col], label=col, color='C{}'.format(j))
            if Nfit is not None:
                axarr[i].plot(Nfitm.index, Nfitm[col], '--', color='C{}'.format(j))
        axarr[i].set(xlabel='$z$', ylabel='$N(z)/N(z={:.1f})$'.format(z0),
                  ylim=(0, 0.35), title='$\log(M/M_\odot) \sim {}$'.format(mdata[i]))
        axarr[i].legend(loc=1, ncol=3)
    plt.tight_layout()

In [ ]:
plot_N(Ndata, Nerr)

In [ ]:
def plot_F(F, Ferr, Ffit=None):
    fig, axarr = plt.subplots(1, 2, figsize=(12, 5))
    axarr = axarr.flat
    Fm = F.filter(regex=('.*R.D,[1234].*'))
    Ferrm = Ferr.filter(regex=('.*R.D,[1234].*'))
    if Ffit is not None:
        Ffitm = Ffit.filter(regex=('.*R.D,[1234].*'))        
    for j, col in enumerate(Fm):
        axarr[0].errorbar(Fm.index, Fm[col], Ferrm[col], label=col, color='C{}'.format(j))
        if Ffit is not None:
            axarr[0].plot(Ffitm.index, Ffitm[col], '--', color='C{}'.format(j))
    axarr[0].set(xlabel='$z$', ylabel='$f_{R|D}(z)$',
              ylim=(0, 0.4))
    axarr[0].legend(loc=1, ncol=4)
    Fm = F.filter(regex=('.*D.R,[1234].*'))
    Ferrm = Ferr.filter(regex=('.*D.R,[1234].*'))
    if Ffit is not None:
        Ffitm = Ffit.filter(regex=('.*D.R,[1234].*'))        
    for j, col in enumerate(Fm):
        axarr[1].errorbar(Fm.index, Fm[col], Ferrm[col], label=col, color='C{}'.format(j))
        if Ffit is not None:
            axarr[1].plot(Ffitm.index, Ffitm[col], '--', color='C{}'.format(j))
    axarr[1].set(xlabel='$z$', ylabel='$f_{D|R}(z)$',
              ylim=(0, 0.4))
    axarr[1].legend(loc=1, ncol=4)
    plt.tight_layout()

In [ ]:
plot_F(Fdata, Ferr)

In [ ]:
class logProbabilities:
    def __init__(self, fit_N0, fit_rates, fit_rates_evol, Nmask,
                 Ndata, Nerr, tdata, ratefn, N0_init, rates_init, rates_evol_init,
                 t0, N_labels, rate_labels, rate_evol_labels):
        # a boolean matrix describing which rates and evolutions to fit
        self.fit_N0 = fit_N0.copy()
        self.nFitN0 = self.fit_N0.values.sum()
        self.fit_rates = fit_rates.copy()
        self.nStates = len(fit_rates)
        self.nFitRates = self.fit_rates.values.sum()
        if rates_evol_init is not None:
            self.fit_rates_evol = fit_rates_evol.copy()
            self.nFitRatesEvol = self.fit_rates_evol.values.sum()
        else:
            self.fit_rates_evol = None
            self.nFitRatesEvol = 0
        self.pars = []
        self.pars.extend(N_labels.values[self.fit_N0.values])
        self.pars.extend(rate_labels.values[self.fit_rates.values])
        if rates_evol_init is not None:
            self.pars.extend(rate_evol_labels.values[self.fit_rates_evol.values])
        self.pars = pd.DataFrame(self.pars, columns=['parameter'])
        self.nPar = self.nFitN0 + self.nFitRates + self.nFitRatesEvol
        # the measured number density, Ndata,
        # in each state at time, tdata,
        # with its error, Nerr
        self.Ndata = Ndata.copy()
        self.Nerr = Nerr.copy()        
        self.tdata = tdata.copy()
        # the current rate matrix is determined using ratefn
        # from N0, the number densities at t0; rates, the rates at t0; and rates_evol
        self.ratefn = ratefn
        self.N0 = N0_init.copy()
        self.rates = rates_init.copy()
        if rates_evol_init is not None:
            self.rates_evol = rates_evol_init.copy()
        else:
            self.rates_evol = None
        self.t0 = t0
        # Which states are actually measured
        self.Nmask = Nmask

    def split_pars(self, p):
        start, end = (0, self.nFitN0)
        p_N0 = p[start:end]
        start, end = (end, end + self.nFitRates)
        p_rates = p[start:end]
        start, end = (end, end + self.nFitRatesEvol)
        p_rates_evol = p[start:end]
        return p_N0, p_rates, p_rates_evol
        
    def evaluate(self, t, p):
        # update the model quantities given the current parameters
        p_N0, p_rates, p_rates_evol = self.split_pars(p)
        self.N0.values[self.fit_N0.values] = p_N0
        self.rates.values[self.fit_rates.values] = p_rates
        if self.rates_evol is not None:
            self.rates_evol.values[self.fit_rates_evol.values] = p_rates_evol
        # compute N at each tdata, given current parameters
        N0 = self.N0
        rates = self.rates     
        if self.rates_evol is None:
            rates_evol = None
        else:
            rates_evol = self.rates_evol.values
        N = odeint(model, N0, t,
                   args=(self.ratefn, rates, rates_evol, self.t0))
        # strip off unmeasured quantitites
        N = N[:, self.Nmask]
        return N
    
    def prior(self, inpars):
        inshape = inpars.shape
        inpars = inpars.reshape((-1, self.nPar))
        lnP = np.zeros(inpars.shape[0])
        for i, p in enumerate(inpars):
            p_N0, p_rates, p_rates_evol = self.split_pars(p)
            lnP[i] = stats.uniform(0, 1).logpdf(p_N0).sum()
            lnP[i] += stats.uniform(0, 1).logpdf(p_rates).sum()
            lnP[i] += norm_logpdf(p_rates_evol, 0, 0.1).sum()
        return lnP.reshape(inshape[:-1])
    
    def likelihood(self, inpars):
        inshape = inpars.shape
        inpars = inpars.reshape((-1, self.nPar))
        lnL = np.zeros(inpars.shape[0])
        for i, p in enumerate(inpars):
            N = self.evaluate(self.tdata, p)
            # calculate Likelihood of computed N given Ndata and Nerr
            lnL[i] = norm_logpdf(N, self.Ndata.values, self.Nerr.values).sum()
        return lnL.reshape(inshape[:-1])
    
    def posterior(self, inpars):
        lnL = self.likelihood(inpars)
        lnP = self.prior(inpars)
        return lnL + lnP
    
    def neg_posterior(self, inpars):
        return -self.posterior(inpars)
    
    def init_pars(self, shape):
        ip = np.zeros(shape + (self.nPar,))
        start, end = (0, self.nFitN0)
        ip[..., start:end] = stats.uniform.rvs(0, 1, size=shape + (self.nFitN0,))
        start, end = (end, end + self.nFitRates)
        ip[..., start:end] = stats.uniform.rvs(0, 1, size=shape + (self.nFitRates,))
        start, end = (end, end + self.nFitRatesEvol)
        ip[..., start:end] = stats.norm.rvs(0, 0.1, size=shape + (self.nFitRatesEvol,))
        return ip

In [ ]:
# Define initial values.
# Those to be fit will vary.
# The rest will be held fixed at their initial values.
N0_init = pd.Series(index=N_labels, data=N0, name=False)
rates_init = pd.DataFrame(index='to ' + states, columns='from ' + states,
                          data=np.zeros((nStates, nStates)))
#rates_evol_init = pd.DataFrame(index='to ' + states, columns='from ' + states,
#                               data=np.zeros((nStates, nStates)))
rates_evol_init = None

In [ ]:
N0_init

In [ ]:
rates_init

In [ ]:
rates_evol_init

In [ ]:
Nmask = np.array(['1' in l or '2' in l or '3' in l or '4' in l for l in N_labels])

In [ ]:
logProb = logProbabilities(fit_N0, fit_rates, fit_rates_evol, Nmask, Ndata, Nerr, tdata,
                           rates_function, N0_init, rates_init, rates_evol_init,
                           t0, N_labels, rate_labels, rate_evol_labels)

In [ ]:
logProb.pars

In [ ]:
logProb.posterior(logProb.init_pars((5,)))

In [ ]:
start = logProb.init_pars((1,))[0]
start[:12] = Ndata.values[0]
start[12:] = 0.00

In [ ]:
logProb.posterior(np.array(start))

In [ ]:
result = scipy.optimize.minimize(logProb.neg_posterior, start, tol=1e-4, method='Nelder-Mead')
result

In [ ]:
Nfit = Ndata.copy()
Nfit[:] = logProb.evaluate(tdata, result.x)

In [ ]:
plot_N(Ndata, Nerr, Nfit)

In [ ]:
Ffit = calc_F(Nfit)

In [ ]:
plot_F(Fdata, Ferr, Ffit)

In [ ]:
sampler = run_emcee(logProb.likelihood, logProb.prior, logProb.init_pars,
                    ntemps=3, nwalkers=100, nsamples=1500,
                    minlogbeta=None, nupdates=150, threads=1,
                    outfilename='mel_bins_data', saveall=True)

In [ ]:
stats = plots.print_emcee(sampler, logProb.pars['parameter'], nburn=750,
                          model=logProb.evaluate, x=tdata, y=Ndata, yerror=Nerr,
                          outfilename='mel_bins_data')